## Visualization script for multiplayer Type Through the Bible files

By Ken Burchfiel

Released under the MIT license

Note: this script, though developed as a Jupyter Notebook to make development and testing easier, is meant to be called as a Python file within a terminal/command prompt with a multiplayer test results file as its sole argument. Here's an example of what this function call may look like on Linux when the current directory is TTTB's build/ folder:

    python mp_visualizations.py 20250706T220148

Note that only the timestamp component of the multiplayer file(s) is passed to the argument; the function will take care of the rest. (This helps prevent user-submitted strings from getting passed to a system() call, which could potentially cause security issues.

When players complete multiplayer games within Type Through The Bible, this script will then get called via a system() call. However, it can also be run as a standalone file if needed.

In [1]:
import time
start_time = time.time()
import os
import pandas as pd
import numpy as np
import plotly.express as px

# The following file paths are relative to the build folder.
mp_results_folder = '../Files/Multiplayer/'
mp_visualizations_folder = '../Visualizations/Multiplayer/'

Checking whether this script is running within a Jupyter notebook: 

(This will allow us to determine whether to specify our multiplayer filename via argparse (which will only work when the .py version of the file is being rurn) or via a notebook-specific argument.

In [2]:
notebook_exec = False
try:
    get_ipython()
    notebook_exec = True # Script is running within a notebook
except:
#     print("get_ipython() failed, so we'll assume that we're running \
# this script within a .py file.")
    pass

# notebook_exec
    

Adding code that will allow the caller to specify which multiplayer results file to analyze:

(Note that only the timestamp component of this file should be passed to the argument.)

If you're running this code within a notebook, update the test_results_timestamp value as needed.

In [3]:
if notebook_exec == False:
    # The following code was based on
    # https://docs.python.org/3/howto/argparse.html#argparse-tutorial
    # and # https://docs.python.org/3/library/argparse.html .
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("results_timestamp")
    args = parser.parse_args()
    test_results_timestamp = args.results_timestamp
else:
    test_results_timestamp = '20250706T220148'

test_results_timestamp

'20250706T220148'

Determining the multiplayer test_results.csv file whose timestamp matches our test_results_timestamp string:

(There *should* only be one such timestamp, but just in case two or more share this timestamp--which is extremely improbable--only one will be retained.)

In [4]:
test_results_file = [file for file in os.listdir(mp_results_folder) if 
 (test_results_timestamp in file) & ('test_results' in file)][0]
# Creating a shortened version of this string that doesn't have
# the 'test_results.csv' component at the end: 
# (This shortened version will serve as the initial component
# of our visualization filenames.)
test_results_name = test_results_file[:-17]
test_results_file, test_results_name

('20250706T220148_string_test_results.csv', '20250706T220148_string')

In [5]:
df = pd.read_csv(mp_results_folder+test_results_file) 
# Replacing the tag columns with their actual meanings:
df.rename(columns = {'Tag_1':'Round','Tag_2':'Test within round',
                     'Tag_3':'Game test number'}, inplace = True)
# Making sure that the results are stored in the order that they were
# typed:
df.sort_values('Game test number', inplace = True)

# Adding a column that shows round and test combinations for each race:
# (This can be used as an x axis value for our WPM-by-race graph.)
df['Round_Test'] = df['Round'].astype('str') + '_' + df[
'Test within round'].astype('str')
# Adding another column that shows how many tests each player has completed
# so far:
# (cumcount starts at 0, so we'll need to add 1 to this value for it
# to work. Also note that the DataFrame doesn't need to be sorted by 
# player beforehand in order for this code to work correctly.)
df['Player test number'] = df.groupby(
    'Player')['WPM'].transform('cumcount') + 1
# Calculating cumulative WPM values:
df['Cumulative WPM'] = (df.groupby('Player')['WPM'].transform(
'cumsum')) / df['Player test number']

df['Best Round_Test WPM'] = df.groupby(
    'Round_Test')['WPM'].transform('max')
df['Player had best WPM for this test'] = np.where(
    df['WPM'] == df['Best Round_Test WPM'], 1, 0)

df.tail()

,Unix_Test_Start_Time,Local_Test_Start_Time,Unix_Test_End_Time,Local_Test_End_Time,Verse_ID,Verse_Code,Verse,Characters,WPM,Test_Seconds,...,Marathon_Mode,Player,Round,Test within round,Game test number,Round_Test,Player test number,Cumulative WPM,Best Round_Test WPM,Player had best WPM for this test
35,1751855137,2025-07-06T22:25:37-0400,1751855152,2025-07-06T22:25:52-0400,35015,James_1:12,Blessed is the man who suffers temptation. For...,152,125.030289,14.588465,...,0,Kendog,4,1,36,4_1,16,118.917722,125.030289,1
36,1751855154,2025-07-06T22:25:54-0400,1751855166,2025-07-06T22:26:06-0400,35016,James_1:13,"No one should say, when he is tempted, that he...",134,136.752611,11.758459,...,0,Kendog,4,2,37,4_2,17,119.966833,136.752611,1
37,1751855166,2025-07-06T22:26:06-0400,1751855175,2025-07-06T22:26:15-0400,35017,James_1:14,"Yet truly, each one is tempted by his own desi...",86,128.508275,8.030611,...,0,Kendog,4,3,38,4_3,18,120.441358,128.508275,1
38,1751855177,2025-07-06T22:26:17-0400,1751855188,2025-07-06T22:26:28-0400,35018,James_1:15,"Thereafter, when desire has conceived, it give...",122,129.246581,11.327185,...,0,Kendog,4,4,39,4_4,19,120.904790,129.246581,1
39,1751855189,2025-07-06T22:26:29-0400,1751855195,2025-07-06T22:26:35-0400,35019,James_1:16,"And so, do not choose to go astray, my most be...",61,124.189468,5.894220,...,0,Kendog,4,5,40,4_5,20,121.069024,124.189468,1


Creating a melted verison of this DataFrame that stores both cumulative and test-specific WPM values within the same column:

(This will make it easier to produce a graph that uses different line dash types to differentiate between cumulative and test-specific WPM values.)

In [6]:
df_wpm_type_melt = df.melt(id_vars = ['Player', 'Round_Test', 
'Player test number'],
value_vars = ['WPM', 'Cumulative WPM'],
var_name = 'WPM Type',
value_name = 'Words per minute').rename(
    columns = {'Words per minute':'WPM'}) # This column couldn't be 
# initialized as WPM because a column with that name was already present 
# within df.
df_wpm_type_melt['WPM Type'] = df_wpm_type_melt['WPM Type'].replace(
{'WPM':'Test WPM'})

df_wpm_type_melt

,Player,Round_Test,Player test number,WPM Type,WPM
0,Alliecat,1_1,1,Test WPM,83.005288
1,Alliecat,1_2,2,Test WPM,32.975491
2,Alliecat,1_3,3,Test WPM,63.254496
3,Alliecat,1_4,4,Test WPM,61.654022
4,Alliecat,1_5,5,Test WPM,72.645656
...,...,...,...,...,...
75,Kendog,4_1,16,Cumulative WPM,118.917722
76,Kendog,4_2,17,Cumulative WPM,119.966833
77,Kendog,4_3,18,Cumulative WPM,120.441358
78,Kendog,4_4,19,Cumulative WPM,120.904790


In [7]:
fig_wpm_by_player = px.line(df_wpm_type_melt, 
x = 'Player test number', y = 'WPM',
        color = 'Player', line_dash = 'WPM Type',
       title = 'WPM By Player and Test',
        color_discrete_sequence = px.colors.qualitative.Alphabet)
fig_wpm_by_player.write_html(
    f'{mp_visualizations_folder}{test_results_name}_Mean_WPM_By_\
Player_And_Test.html',
    include_plotlyjs = 'cdn')
# Note: 'Alphabet' is used here so that up to 26 distinct colors can be
# shown within the chart (which will prove useful for multiplayer rounds
# with larger player counts).
# fig_wpm_by_player

Calculating mean WPMs by player and round as well as overall WPMs:

In [8]:
df_mean_wpm_by_player_and_round = df.pivot_table(
index = ['Player', 'Round'], values = 'WPM', 
               aggfunc = 'mean').reset_index()

# Adding overall WPMs for each player to the bottom of this DataFrame:

df_mean_wpm_by_player = df.pivot_table(
index = 'Player', values = 'WPM', 
               aggfunc = 'mean').reset_index()
df_mean_wpm_by_player['Round'] = 'Overall'

df_mean_wpm_by_player_and_round = pd.concat([
    df_mean_wpm_by_player_and_round, df_mean_wpm_by_player]).reset_index(
    drop=True)
# Ensuring that all Round values are strings: (I found that, without
# this update, the Overall rows would not appear within the figure
# that we're about to create.)
df_mean_wpm_by_player_and_round['Round'] = (
    df_mean_wpm_by_player_and_round['Round'].astype('str'))
df_mean_wpm_by_player_and_round

,Player,Round,WPM
0,Alliecat,1,62.706991
1,Alliecat,2,93.708716
2,Alliecat,3,96.977565
3,Alliecat,4,91.524123
4,Kendog,1,113.541991
5,Kendog,2,122.283725
6,Kendog,3,119.704936
7,Kendog,4,128.745445
8,Alliecat,Overall,86.229349
9,Kendog,Overall,121.069024


In [9]:
fig_mean_wpm_by_player_and_round = px.bar(
    df_mean_wpm_by_player_and_round, x = 'Round', 
       y = 'WPM', color = 'Player', barmode = 'group',
    color_discrete_sequence = px.colors.qualitative.Alphabet,
    title = 'Mean WPM by Player and Round',
      text_auto = '.0f')
fig_mean_wpm_by_player_and_round.write_html(
    f'{mp_visualizations_folder}{test_results_name}_Mean_WPM_\
By_Player_And_Round.html',
    include_plotlyjs = 'cdn')
# fig_mean_wpm_by_player_and_round

In [10]:
df_wins = df.pivot_table(index = 'Player', values = 'Player had best \
WPM for this test', aggfunc = 'sum').reset_index()
df_wins.sort_values('Player had best WPM for this test', ascending = False,
                   inplace = True)
df_wins

,Player,Player had best WPM for this test
1,Kendog,18
0,Alliecat,2


In [11]:
fig_wins = px.bar(df_wins, x = 'Player', 
       y = 'Player had best WPM for this test',
      title = 'Number of Tests in Which Each Player \
Had the Highest WPM', text_auto = '.0f',
                 color = 'Player', 
color_discrete_sequence=px.colors.qualitative.Alphabet).update_layout(
yaxis_title = 'Wins')
fig_wins.write_html(f'{mp_visualizations_folder}{test_results_name}_\
wins_by_player.html', include_plotlyjs = 'cdn')
# fig_wins

In [12]:
df_highest_wpm = df.pivot_table(index = 'Player', values = 'WPM', 
                                aggfunc = 'max').reset_index()
df_highest_wpm.sort_values('WPM', ascending = False, inplace = True)
df_highest_wpm

,Player,WPM
1,Kendog,174.177995
0,Alliecat,108.959206


In [13]:
fig_highest_wpm = px.bar(df_highest_wpm, x = 'Player', 
       y = 'WPM',
      title = 'Highest WPM by Player', text_auto = '.3f',
                 color = 'Player', 
                 color_discrete_sequence = 
px.colors.qualitative.Alphabet)
fig_highest_wpm.write_html(
    f'{mp_visualizations_folder}{test_results_name}_\
highest_wpm_by_player.html', include_plotlyjs = 'cdn')
# fig_highest_wpm

In [14]:
end_time = time.time()
run_time = end_time - start_time
print(f"Finished calculating and visualizing multiplayer stats in \
{round(run_time, 3)} seconds.")

Finished calculating and visualizing multiplayer stats in 0.688 seconds.
